import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

In [2]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

def wndwng0d(x,snstvty=2):            #روش تشخیص ایپاک پایه
    smpl_rte=480
    i=0
    wndws=np.array([])
    #for i in range (1,2):   #print(np.shape(x))
    if snstvty==1 :                        #تشخیص ایپاک پایه
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=200,prominence=300, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=160,prominence=100, width=3)
            #print('class ', i, 'Row = ', j, 'have ',len(peaks),'peaks')
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :  #win=np.append(win, i)# if len(win)==smpl_rte+1 :
                        wndws=np.append(wndws, win)


    if snstvty==2 :                        #تشخیص ایپاک حساس تر
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.02*max(xj), distance=160,prominence=200, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.01*max(xj), distance=120,prominence=60, width=3)
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :
                        wndws=np.append(wndws, win)
    rows=int(len(wndws)/(smpl_rte))            #/(smpl_rte+1))
    wndws0=np.reshape(wndws,(rows,(smpl_rte)))    #(wndws,(rows,(smpl_rte+1)))
    return(wndws0)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=17
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)]=MITBIH(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=200

1 283	2 66	3 20	4 135	5 13	6 21	7 133	8 55	9 13	10 10	11 10	12 10	13 11	14 103	15 62	16 10	17 45	

In [4]:
np.shape(ecg)

(1000, 3601)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train&Test windows</div></center></h1>

In [5]:
smpl_rte=480              # Should be equal with modules (=480)
windws=np.array([])
tst_raw=np.empty((0,3601), float) 
for cls in range (1,cls_num+1):                                                      #ساخت پنجره های داده های آموزش اصلی
    dta=np.array(vars()['ecg'+str(cls)][int(.1*len(vars()['ecg'+str(cls)])):,:-1])     #انتخاب محدوده ترِین جهت داده افزایی
    dta_tst=np.array(vars()['ecg'+str(cls)][:int(.1*len(vars()['ecg'+str(cls)])),:-1]) #انتخاب محدوده تست جهت داده افزایی
    vars()['wndws'+str(cls)]=wndwng(dta)                
    vars()['ecg_test'+str(cls)]=wndwng(dta_tst)        
    cls_clm0=np.zeros(len(vars()['wndws'+str(cls)]))
    cls_clm0_tst=np.zeros(len(vars()['ecg_test'+str(cls)]))
    #cls_clm0=np.array(cls_clm0)
    #cls_clm_tst=np.array(cls_clm0_tst)
    cls_clm=np.int16(cls_clm0+cls)
    cls_clm_tst=np.int16(cls_clm0_tst+cls)
    cls_clm=np.transpose([cls_clm])                     # add class labels
    cls_clm_tst=np.transpose([cls_clm_tst])             # add class labels
    vars()['wndws'+str(cls)]=np.concatenate((vars()['wndws'+str(cls)],cls_clm),axis=1)   
    vars()['ecg_test'+str(cls)]=np.concatenate((vars()['ecg_test'+str(cls)],cls_clm_tst),axis=1)   
    tst_raw=np.append(tst_raw,np.array(vars()['ecg'+str(cls)][:int(.1*len(vars()['ecg'+str(cls)])),:]),axis=0)

#Merge Classes
xtrain=np.empty((0,len(wndws1[0])), float) 
xtest=np.empty((0,len(wndws1[0])), float)
for i in range (1,cls_num+1):
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    xtest=np.append(xtest,vars()['ecg_test'+str(i)],axis=0)
    #print('Up to class ', i, 'train shape = ', np.shape(xtrain))
    #print('Up to class ', i, 'test shape = ', np.shape(xtest)) 
    
print("each class and its windows =")
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                         # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
cls_wndws=np.int16(cls_wndws)    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 2598	2 788	3 224	4 1563	5 158	6 189	7 1369	8 552	9 114	10 103	11 72	12 107	13 156	14 1011	15 632	16 45	17 450	
 max = 2598


In [6]:
#Normalization
mx_aug=np.max(xtrain[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
xtest[:,:-1]= 2*(xtest[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #
print(xtrain[:,-1])
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtest[:,:-1]))

[   0 2598  788  224 1563  158  189 1369  552  114  103   72  107  156
 1011  632   45  450]
[ 1.  1.  1. ... 17. 17. 17.]
min trn = -1.0
max trn = 0.6348682809475628


<h3><center><div style="direction:rtl;font-family:B Nazanin">Seprate labels</div></center></h1>

In [8]:
wndws_test=np.array(xtest)
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [9]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

In [10]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (10131, 480)
y_train =>  (10131,)
X_test  =>  (1114, 480)
y_test  =>  (1114,)
X_valid  =>  (10131, 480)
y_valid  =>  (10131,)


<h1><center><div style="direction:rtl;font-family:B Nazanin">Resnext 18 layer CNN for segmented signal prediction</div></center></h1>

In [11]:
y_train=np.array(y_train-1)
y_test=np.array(y_test-1)
y_valid=np.array(y_valid-1)

In [12]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [13]:
'''#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 200
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data
'''
# Import ResNet1D Module
#from ResNet_1DCNN import ResNet
#from ResNeXt_1DCNN import ResNeXt as ResNet
from SE_ResNeXt_1DCNN import SEResNeXt as ResNet

"Configurations for ResNet in ResNext Mode"
length = X_train.shape[1]   # Number of Features (or length of the signal)
model_width = 32           # Number of Filter or Kernel in the Input Layer
num_channel = 1             # Number of Input Channels
problem_type = 'Classification' # ResNext or Classification
output_number = cls_num     # Number of Outputs in the ResNext Mode
ResNext18_Model = ResNet(length, num_channel, model_width, problem_type=problem_type, output_nums=output_number).SEResNeXt18() # Build Model
# SEResNext Models supported: SEResNeXt18, SEResNeXt34, SEResNeXt50, SEResNeXt101, SEResNeXt152, 
ResNext18_Model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 100, restore_best_weights = True)
history = ResNext18_Model.fit(X_train, y_train_cat, epochs=500, batch_size=btch, verbose=1, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping]) #,validation_split=0.2,callbacks =[callbacks]

Epoch 1/500
51/51 [==============================] - 27s 234ms/step - loss: 1.2184 - accuracy: 0.6175 - val_loss: 2.6511 - val_accuracy: 0.2564
Epoch 2/500
51/51 [==============================] - 10s 189ms/step - loss: 0.6303 - accuracy: 0.7851 - val_loss: 2.7540 - val_accuracy: 0.2564
Epoch 3/500
51/51 [==============================] - 10s 189ms/step - loss: 0.4715 - accuracy: 0.8304 - val_loss: 2.9538 - val_accuracy: 0.2405
Epoch 4/500
51/51 [==============================] - 10s 189ms/step - loss: 0.3972 - accuracy: 0.8551 - val_loss: 3.0839 - val_accuracy: 0.2114
Epoch 5/500
51/51 [==============================] - 10s 189ms/step - loss: 0.3357 - accuracy: 0.8768 - val_loss: 3.4159 - val_accuracy: 0.2459
Epoch 6/500
51/51 [==============================] - 10s 189ms/step - loss: 0.2960 - accuracy: 0.8852 - val_loss: 3.1821 - val_accuracy: 0.2469
Epoch 7/500
51/51 [==============================] - 10s 190ms/step - loss: 0.2707 - accuracy: 0.8909 - val_loss: 3.1513 - val_accuracy:

51/51 [==============================] - 10s 189ms/step - loss: 0.0391 - accuracy: 0.9862 - val_loss: 1.1243 - val_accuracy: 0.8108
Epoch 58/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0230 - accuracy: 0.9923 - val_loss: 0.2075 - val_accuracy: 0.9466
Epoch 59/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0309 - accuracy: 0.9910 - val_loss: 0.1455 - val_accuracy: 0.9555
Epoch 60/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0293 - accuracy: 0.9905 - val_loss: 0.1601 - val_accuracy: 0.9533
Epoch 61/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0313 - accuracy: 0.9891 - val_loss: 1.8998 - val_accuracy: 0.7000
Epoch 62/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0330 - accuracy: 0.9895 - val_loss: 1.0397 - val_accuracy: 0.7995
Epoch 63/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0354 - accuracy: 0.9875 - val_loss: 0.7211 - val_accuracy: 0.852

51/51 [==============================] - 10s 188ms/step - loss: 0.0157 - accuracy: 0.9952 - val_loss: 0.0531 - val_accuracy: 0.9829
Epoch 114/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0170 - accuracy: 0.9956 - val_loss: 0.1671 - val_accuracy: 0.9573
Epoch 115/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0103 - accuracy: 0.9967 - val_loss: 0.2581 - val_accuracy: 0.9431
Epoch 116/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0137 - accuracy: 0.9957 - val_loss: 0.2548 - val_accuracy: 0.9454
Epoch 117/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0149 - accuracy: 0.9953 - val_loss: 0.0687 - val_accuracy: 0.9816
Epoch 118/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0160 - accuracy: 0.9952 - val_loss: 0.0646 - val_accuracy: 0.9808
Epoch 119/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0166 - accuracy: 0.9947 - val_loss: 0.1478 - val_accuracy:

51/51 [==============================] - 10s 189ms/step - loss: 0.0079 - accuracy: 0.9972 - val_loss: 0.8526 - val_accuracy: 0.8578
Epoch 170/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0098 - accuracy: 0.9965 - val_loss: 0.5500 - val_accuracy: 0.9032
Epoch 171/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0112 - accuracy: 0.9960 - val_loss: 0.5456 - val_accuracy: 0.8947
Epoch 172/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0126 - accuracy: 0.9966 - val_loss: 0.4832 - val_accuracy: 0.9031
Epoch 173/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0114 - accuracy: 0.9969 - val_loss: 0.1430 - val_accuracy: 0.9681
Epoch 174/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0100 - accuracy: 0.9969 - val_loss: 0.0935 - val_accuracy: 0.9747
Epoch 175/500
51/51 [==============================] - 10s 189ms/step - loss: 0.0069 - accuracy: 0.9980 - val_loss: 0.5984 - val_accuracy:

51/51 [==============================] - 10s 190ms/step - loss: 0.0062 - accuracy: 0.9982 - val_loss: 1.1542 - val_accuracy: 0.8287
Epoch 226/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0111 - accuracy: 0.9976 - val_loss: 0.2282 - val_accuracy: 0.9511
Epoch 227/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0058 - accuracy: 0.9981 - val_loss: 0.6935 - val_accuracy: 0.8962
Epoch 228/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0066 - accuracy: 0.9976 - val_loss: 0.0924 - val_accuracy: 0.9804
Epoch 229/500
51/51 [==============================] - 10s 191ms/step - loss: 0.0101 - accuracy: 0.9968 - val_loss: 0.0123 - val_accuracy: 0.9962
Epoch 230/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0083 - accuracy: 0.9979 - val_loss: 0.1975 - val_accuracy: 0.9531
Epoch 231/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.5271 - val_accuracy:

51/51 [==============================] - 10s 190ms/step - loss: 0.0054 - accuracy: 0.9983 - val_loss: 0.0453 - val_accuracy: 0.9869
Epoch 282/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0070 - accuracy: 0.9979 - val_loss: 0.2425 - val_accuracy: 0.9466
Epoch 283/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0040 - accuracy: 0.9989 - val_loss: 0.0243 - val_accuracy: 0.9934
Epoch 284/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0068 - accuracy: 0.9976 - val_loss: 0.5098 - val_accuracy: 0.9096
Epoch 285/500
51/51 [==============================] - 10s 191ms/step - loss: 0.0037 - accuracy: 0.9988 - val_loss: 1.4149 - val_accuracy: 0.8430
Epoch 286/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0048 - accuracy: 0.9982 - val_loss: 0.8334 - val_accuracy: 0.8880
Epoch 287/500
51/51 [==============================] - 10s 190ms/step - loss: 0.0046 - accuracy: 0.9990 - val_loss: 1.3080 - val_accuracy:

51/51 [==============================] - 10s 188ms/step - loss: 0.0064 - accuracy: 0.9982 - val_loss: 0.2020 - val_accuracy: 0.9539
Epoch 338/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0060 - accuracy: 0.9979 - val_loss: 0.1721 - val_accuracy: 0.9620
Epoch 339/500
51/51 [==============================] - 10s 187ms/step - loss: 0.0026 - accuracy: 0.9990 - val_loss: 0.0348 - val_accuracy: 0.9909
Epoch 340/500
51/51 [==============================] - 10s 187ms/step - loss: 0.0025 - accuracy: 0.9992 - val_loss: 0.1813 - val_accuracy: 0.9609
Epoch 341/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0049 - accuracy: 0.9987 - val_loss: 0.0700 - val_accuracy: 0.9820
Epoch 342/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0030 - accuracy: 0.9993 - val_loss: 0.3250 - val_accuracy: 0.9311
Epoch 343/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0035 - accuracy: 0.9987 - val_loss: 0.0315 - val_accuracy:

51/51 [==============================] - 10s 187ms/step - loss: 0.0033 - accuracy: 0.9995 - val_loss: 0.0843 - val_accuracy: 0.9796
Epoch 394/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0028 - accuracy: 0.9996 - val_loss: 0.0071 - val_accuracy: 0.9975
Epoch 395/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0041 - accuracy: 0.9987 - val_loss: 0.0173 - val_accuracy: 0.9950
Epoch 396/500
51/51 [==============================] - 10s 187ms/step - loss: 0.0029 - accuracy: 0.9991 - val_loss: 0.1187 - val_accuracy: 0.9751
Epoch 397/500
51/51 [==============================] - 10s 187ms/step - loss: 0.0024 - accuracy: 0.9996 - val_loss: 0.0128 - val_accuracy: 0.9962
Epoch 398/500
51/51 [==============================] - 10s 187ms/step - loss: 0.0047 - accuracy: 0.9994 - val_loss: 0.1869 - val_accuracy: 0.9663
Epoch 399/500
51/51 [==============================] - 10s 187ms/step - loss: 0.0043 - accuracy: 0.9985 - val_loss: 0.0130 - val_accuracy:

51/51 [==============================] - 10s 188ms/step - loss: 0.0053 - accuracy: 0.9986 - val_loss: 0.0415 - val_accuracy: 0.9908
Epoch 450/500
51/51 [==============================] - 10s 187ms/step - loss: 0.0035 - accuracy: 0.9988 - val_loss: 0.0602 - val_accuracy: 0.9865
Epoch 451/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0058 - accuracy: 0.9990 - val_loss: 0.2387 - val_accuracy: 0.9581
Epoch 452/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0045 - accuracy: 0.9983 - val_loss: 0.0179 - val_accuracy: 0.9954
Epoch 453/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0062 - accuracy: 0.9984 - val_loss: 0.3497 - val_accuracy: 0.9385
Epoch 454/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0062 - accuracy: 0.9988 - val_loss: 3.3715 - val_accuracy: 0.7236
Epoch 455/500
51/51 [==============================] - 10s 188ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 0.0171 - val_accuracy:

In [14]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [15]:
predict_x=ResNext18_Model.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                 # function from augment.py to remove 0 index predictions

#y_pred_crs = model_Res.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
print(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

35/35 [==============================] - 2s 28ms/step
[[ 73  53   1   2   3   0  84  36   9   0   0   0  43   0   5   0   0]
 [ 10  18   0   0   3   0  13   5   1   0   0   0  17   0   0   0   0]
 [  3   0   0  25   0   0   2   2   1   0   1   0   0   0   0   0   0]
 [ 34   1  10  32   0   0  14  48   0   0  12   2   0   0   0   0   0]
 [  3   0   0   0   0   0   6   2   0   1   0   0   0   6   0   0   0]
 [  0   0   0   0   0  17   3   0   0   0   0   0   0   0   0   0   0]
 [ 98   2   0   0   0   1  53   2   0   0   0   0   3   0   1   0   0]
 [  2   0   0   0   0   0  12  43   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   3   2   3   0   0   0   0   0   1   0   0]
 [  0   0   0   0   0   0   7   2   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0   3   1   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0  15   0   0   0   0   0]
 [  2   0   0   0   0   0   6   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  

<h3><center><div style="direction:rtl;font-family:B Nazanin">Long Term main ECG signal Prediction</div></center></h1>

In [18]:
ecg_test=np.array(xtest)
i=0
mlpinputshps=np.array([])
maxcntlbl=np.array([])
mlpinput=np.array([])
mlplbl=np.array([])
cnnlbl=np.array([])
peaklens=np.array([])            #array of peaks quantity in each raw signal
x=np.array([])
wow=0

for i in range (len(ecg_test)): #calculating max peaks in row signals
    peaklens=np.append(peaklens,len(wndwng0d(ecg_test[i])))
maxpeaks=int(np.max(peaklens))

for i in range (len(tst_raw)):
    windws=np.array([])
    yyy=np.array([])
    yy=np.array([])
    win=np.array([])
    x=np.array(tst_raw[i,:-1])
    windws=wndwng0d(x)
    windws= 2*(windws-mn_aug)/(mx_aug - mn_aug) - 1
    windws = np.expand_dims(windws, axis=2)
    yy= ResNext18_Model.predict(windws,verbose='0')    #yy= Predict each window lable for a raw signal (3600 Pnt) (17 prob in row)
    yyy=np.int16(maxindx(yy))                    # one-hot to class lable
    maxcntlbl=np.append(maxcntlbl,(np.bincount(yyy).argmax()))
maxcntlbl=np.int16(maxcntlbl)
lbls=np.int16(tst_raw[:,-1]-1)

cm = confusion_matrix(lbls, maxcntlbl)
print(cm)
acc3=accuracy_score(lbls, maxcntlbl)
print(classification_report(lbls, maxcntlbl))

[[ 0  5  0  0  0  0 14  3  1  0  0  0  5  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  1  8  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4]]
              precision    recall  f1-score   support

           0       0.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<h1><center><div style="direction:rtl;font-family:B Nazanin">Resnext 50 Layer CNN for segmented signal prediction</div></center></h1>

In [19]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80

from SE_ResNeXt_1DCNN import SEResNeXt as ResNet

"Configurations for ResNet in ResNext Mode"
length = X_train.shape[1]   # Number of Features (or length of the signal)
model_width = 32           # Number of Filter or Kernel in the Input Layer
num_channel = 1             # Number of Input Channels
problem_type = 'Classification' # ResNext or Classification
output_number = cls_num     # Number of Outputs in the ResNext Mode
ResNext50_Model = ResNet(length, num_channel, model_width, problem_type=problem_type, output_nums=output_number).SEResNeXt50() # Build Model
# SEResNext Models supported: SEResNeXt18, SEResNeXt34, SEResNeXt50, SEResNeXt101, SEResNeXt152, 
ResNext50_Model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 100, restore_best_weights = True)
history = ResNext50_Model.fit(X_train, y_train_cat, epochs=500, batch_size=btch, verbose=1, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping]) #,validation_split=0.2,callbacks =[callbacks]

Epoch 1/500
51/51 [==============================] - 57s 504ms/step - loss: 1.9493 - accuracy: 0.3824 - val_loss: 2.5349 - val_accuracy: 0.1549
Epoch 2/500
51/51 [==============================] - 23s 443ms/step - loss: 1.2322 - accuracy: 0.5958 - val_loss: 2.7845 - val_accuracy: 0.0796
Epoch 3/500
51/51 [==============================] - 23s 446ms/step - loss: 0.8931 - accuracy: 0.7035 - val_loss: 2.8273 - val_accuracy: 0.2163
Epoch 4/500
51/51 [==============================] - 23s 446ms/step - loss: 0.7145 - accuracy: 0.7548 - val_loss: 3.0025 - val_accuracy: 0.2207
Epoch 5/500
51/51 [==============================] - 23s 445ms/step - loss: 0.6096 - accuracy: 0.7875 - val_loss: 3.2423 - val_accuracy: 0.2402
Epoch 6/500
51/51 [==============================] - 23s 446ms/step - loss: 0.5301 - accuracy: 0.8090 - val_loss: 2.6289 - val_accuracy: 0.2939
Epoch 7/500
51/51 [==============================] - 23s 445ms/step - loss: 0.4762 - accuracy: 0.8259 - val_loss: 2.4548 - val_accuracy:

51/51 [==============================] - 23s 444ms/step - loss: 0.0601 - accuracy: 0.9765 - val_loss: 0.1650 - val_accuracy: 0.9508
Epoch 58/500
51/51 [==============================] - 23s 445ms/step - loss: 0.0685 - accuracy: 0.9763 - val_loss: 0.1142 - val_accuracy: 0.9607
Epoch 59/500
51/51 [==============================] - 23s 446ms/step - loss: 0.0686 - accuracy: 0.9758 - val_loss: 0.0500 - val_accuracy: 0.9820
Epoch 60/500
51/51 [==============================] - 23s 445ms/step - loss: 0.0595 - accuracy: 0.9779 - val_loss: 1.0575 - val_accuracy: 0.7645
Epoch 61/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0591 - accuracy: 0.9792 - val_loss: 0.9678 - val_accuracy: 0.7890
Epoch 62/500
51/51 [==============================] - 23s 445ms/step - loss: 0.0684 - accuracy: 0.9755 - val_loss: 0.1269 - val_accuracy: 0.9592
Epoch 63/500
51/51 [==============================] - 23s 445ms/step - loss: 0.0555 - accuracy: 0.9784 - val_loss: 0.4954 - val_accuracy: 0.881

51/51 [==============================] - 23s 444ms/step - loss: 0.0284 - accuracy: 0.9904 - val_loss: 0.2184 - val_accuracy: 0.9500
Epoch 114/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0254 - accuracy: 0.9915 - val_loss: 0.5096 - val_accuracy: 0.8847
Epoch 115/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0240 - accuracy: 0.9927 - val_loss: 0.0769 - val_accuracy: 0.9753
Epoch 116/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0248 - accuracy: 0.9921 - val_loss: 0.0874 - val_accuracy: 0.9722
Epoch 117/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0254 - accuracy: 0.9918 - val_loss: 0.1264 - val_accuracy: 0.9633
Epoch 118/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0239 - accuracy: 0.9917 - val_loss: 0.1357 - val_accuracy: 0.9608
Epoch 119/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0249 - accuracy: 0.9917 - val_loss: 0.0348 - val_accuracy:

51/51 [==============================] - 23s 444ms/step - loss: 0.0186 - accuracy: 0.9949 - val_loss: 0.0329 - val_accuracy: 0.9918
Epoch 170/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0179 - accuracy: 0.9949 - val_loss: 0.0991 - val_accuracy: 0.9757
Epoch 171/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0132 - accuracy: 0.9953 - val_loss: 0.4005 - val_accuracy: 0.9266
Epoch 172/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0179 - accuracy: 0.9947 - val_loss: 0.1419 - val_accuracy: 0.9681
Epoch 173/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0160 - accuracy: 0.9957 - val_loss: 0.3959 - val_accuracy: 0.9271
Epoch 174/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0155 - accuracy: 0.9952 - val_loss: 0.2326 - val_accuracy: 0.9452
Epoch 175/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0162 - accuracy: 0.9961 - val_loss: 0.4672 - val_accuracy:

51/51 [==============================] - 23s 444ms/step - loss: 0.0116 - accuracy: 0.9970 - val_loss: 0.8283 - val_accuracy: 0.8574
Epoch 226/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.0173 - val_accuracy: 0.9958
Epoch 227/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0099 - accuracy: 0.9970 - val_loss: 1.3736 - val_accuracy: 0.7681
Epoch 228/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0113 - accuracy: 0.9965 - val_loss: 0.0274 - val_accuracy: 0.9930
Epoch 229/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0138 - accuracy: 0.9959 - val_loss: 0.0202 - val_accuracy: 0.9933
Epoch 230/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0127 - accuracy: 0.9959 - val_loss: 0.0223 - val_accuracy: 0.9922
Epoch 231/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0086 - accuracy: 0.9967 - val_loss: 0.0746 - val_accuracy:

51/51 [==============================] - 23s 443ms/step - loss: 0.0071 - accuracy: 0.9978 - val_loss: 1.8392 - val_accuracy: 0.7653
Epoch 282/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0070 - accuracy: 0.9986 - val_loss: 0.3619 - val_accuracy: 0.9373
Epoch 283/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0054 - accuracy: 0.9985 - val_loss: 0.0093 - val_accuracy: 0.9973
Epoch 284/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0120 - accuracy: 0.9961 - val_loss: 0.2134 - val_accuracy: 0.9518
Epoch 285/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0069 - accuracy: 0.9975 - val_loss: 0.0789 - val_accuracy: 0.9813
Epoch 286/500
51/51 [==============================] - 23s 444ms/step - loss: 0.0154 - accuracy: 0.9959 - val_loss: 0.0413 - val_accuracy: 0.9883
Epoch 287/500
51/51 [==============================] - 23s 443ms/step - loss: 0.0084 - accuracy: 0.9980 - val_loss: 0.1948 - val_accuracy:

In [20]:
predict_x=ResNext50_Model.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                 # function from augment.py to remove 0 index predictions

#y_pred_crs = model_Res.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
print(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

35/35 [==============================] - 4s 55ms/step
[[ 96  49   0   8  21   0   8  29  41   0   0   0  57   0   0   0   0]
 [  3  22   1   1  11   0   0   1   5   0   0   0  23   0   0   0   0]
 [  0   1  14  19   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  6   0   3 134   2   0   7   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   4   0  14   0   0   0   0   0   0   0]
 [  0   3   0   0   0  16   1   0   0   0   0   0   0   0   0   0   0]
 [123   2   1   3   4   0   9   9   5   0   0   0   3   1   0   0   0]
 [  2   0   0   2   0   0   4  49   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   2   7   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   7   2   0   0   0   0   0   0   0   0   0]
 [  1   0   0   2   0   0   0   0   0   0   1   1   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0  16   0   0   0   0   0]
 [  6   0   0   0   0   0   2   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  

<h3><center><div style="direction:rtl;font-family:B Nazanin">Long Term main ECG signal Prediction</div></center></h1>

In [21]:
        '''ecg_test=np.array(ecg1[:int(.1*len(ecg1)),:])
#print(1,len(ecg_test), end='\t')
for i in range (2,18):
    ecg_test=np.concatenate((ecg_test,vars()['ecg'+str(i)][:int(.1*len(vars()['ecg'+str(i)])),:]),axis=0)
    #print(i,len(vars()['ecg'+str(i)]), end='\t')  '''  
#print('\n ecg_test shape = ',np.shape(ecg_test))
ecg_test=np.array(xtest)
i=0
mlpinputshps=np.array([])
maxcntlbl=np.array([])
mlpinput=np.array([])
mlplbl=np.array([])
cnnlbl=np.array([])
peaklens=np.array([])            #array of peaks quantity in each raw signal
x=np.array([])
wow=0

for i in range (len(ecg_test)): #calculating max peaks in row signals
    peaklens=np.append(peaklens,len(wndwng0d(ecg_test[i])))
maxpeaks=int(np.max(peaklens))
#print(maxpeaks)

#prd_lbls_tst=np.empty((0,maxpeaks), int)              #have peaks(windows) lables for each raw signal

for i in range (len(tst_raw)):
    windws=np.array([])
    yyy=np.array([])
    yy=np.array([])
    win=np.array([])
    x=np.array(tst_raw[i,:-1])
    windws=wndwng0d(x)
    windws= 2*(windws-mn_aug)/(mx_aug - mn_aug) - 1
    windws = np.expand_dims(windws, axis=2)
    yy= ResNext50_Model.predict(windws,verbose='0')    #yy= Predict each window lable for a raw signal (3600 Pnt) (17 prob in row)
    yyy=np.int16(maxindx(yy))                    # one-hot to class lable
    maxcntlbl=np.append(maxcntlbl,(np.bincount(yyy).argmax()))
maxcntlbl=np.int16(maxcntlbl)
lbls=np.int16(tst_raw[:,-1]-1)

cm = confusion_matrix(lbls, maxcntlbl)
print(cm)
acc3=accuracy_score(lbls, maxcntlbl)
print(classification_report(lbls, maxcntlbl))

[[ 0 10  0  0  0  0  4  3  1  0  0  0 10  0  0  0  0]
 [ 0  3  0  0  1  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 12  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  1  1  0 10  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4]]
              precision    recall  f1-score   support

           0       0.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
